In [3]:
!pip install google-api-python-client

In [1]:
from googleapiclient.discovery import build
import pandas as pd


## 크롤링한 사이트
유튜브 [뭉클한 주말🥰 #13] 서울 한복판에서 느끼는 이웃의 따뜻한 정 | 행복을 파는 가게 - 국수 할매와 구두 할배 [다큐 공감 | KBS 130924 방송]<br>
https://www.youtube.com/watch?v=S9lkaozex0w

𝐏𝐥𝐚𝐲𝐥𝐢𝐬𝐭 긍정 에너지 뿜어내는 출근길 노래 모음😄 행복 가득한 하루를 위한 텐션 업 플리🎈<br>
https://www.youtube.com/watch?v=PYUPmD7eQz0

힘들고 지칠때 위로가 되어주는 노래<br>
https://www.youtube.com/watch?v=5ebOrg5Njnc

[Playlist] 괜찮아, 다 잘될거야. 힘들고 지친 당신을 위로해주는 노래🎵<br>
https://www.youtube.com/watch?v=uGa2cN1N5-Y

자신감이 없을 때, 힘들고 지칠 때 힘이 되어주는 노래모음<br>
https://www.youtube.com/watch?v=C57eP02b5f4

[𝐏𝐥𝐚𝐲𝐥𝐢𝐬𝐭] 나 힘들어, 안아줘. 당신의 마음을 위로하는 플레이리스트🩹<br>
https://www.youtube.com/watch?v=ObY22aMUMxE

유튜브 댓글 크롤링

In [2]:
comments = list()
api_key = '' #유튜브 댓글 크롤링 api키
video_id = '' #유튜브 주소 중 '=' 이후 id
api_obj = build('youtube', 'v3', developerKey=api_key)
response = api_obj.commentThreads().list(part='snippet,replies', videoId=video_id, maxResults=100).execute()

while response:
    for item in response['items']:
        comment = item['snippet']['topLevelComment']['snippet']
        comments.append([comment['textDisplay'], comment['authorDisplayName'], comment['publishedAt'], comment['likeCount']])
 
        if item['snippet']['totalReplyCount'] > 0:
            for reply_item in item['replies']['comments']:
                reply = reply_item['snippet']
                comments.append([reply['textDisplay'], reply['authorDisplayName'], reply['publishedAt'], reply['likeCount']])
 
    if 'nextPageToken' in response:
        response = api_obj.commentThreads().list(part='snippet,replies', videoId=video_id, pageToken=response['nextPageToken'], maxResults=100).execute()
    else:
        break

df = pandas.DataFrame(comments, columns=['comment', 'author', 'date', 'num_likes'])
df.drop(columns=['author', 'date'], inplace=True)
df.to_excel('긍정.xlsx', header=['comment', 'num_likes'], index=None)

문장 토큰화

In [5]:
import pandas as pd
from konlpy.tag import Okt

def kor_token(comment):
    okt = Okt()
    token_words = okt.morphs(comment)
    return token_words

# add_token_result = 엑셀 파일을 읽어와 각 행의 문장을 토큰화하고 결과를 새로운 열에 추가하여 반환하는 함수 
def add_token_result(파일경로):
    df_token = pd.read_excel(파일경로)
    list_token = []
    for index, row in df_token.iterrows():
        comment = row['comment']  # 'comment' 열을 토큰화
        token_words = kor_token(comment)
        list_token.append(token_words)
    df_token['result_token'] = list_token
    return df_token


df_token = add_token_result("긍정.xlsx")

new = "긍정토큰화.xlsx"
df_token.to_excel(new, index=False)

한글 감성어 사전(KNU) 업로드

In [2]:
import json
import pandas as pd
with open('SentiWord_info.json', encoding='utf-8-sig', mode='r') as f: 
  SentiWord_info = json.load(f)

sentiword_dic = pd.DataFrame(SentiWord_info)

In [17]:
new = pd.read_excel('긍정토큰화.xlsx')

excel_list = new.iloc[:, 2].tolist()


final = pd.DataFrame(columns=("comment", "sentiment"))  
idx = 0                                             
 
for item in excel_list:                                # 전체 댓글에서 문장 하나씩 가져옴 
  sentiment = 0                                     # 초기 감정 점수값을 0으로 설정
  for i in range(0, len(sentiword_dic)):            # 감성사전의 모든 단어를 하나씩 선택
    if sentiword_dic.word[i] in item:              # 댓글 문장에 감성 단어가 있는지 확인
      sentiment += int(sentiword_dic.polarity[i])   # 감성단어가 있다면 감정 점수값 합계를 구함.
  final.loc[idx] = [item, sentiment]                  # 댓글별 감정 점수값을 데이터프레임으로 쌓음
  idx += 1

final['original_comment'] = new.iloc[:, 0]  #원래 댓글 추가하기

final.to_excel("긍정결과.xlsx", index=False)
